In [61]:
# Python imports
from pathlib import Path

# plot imports
import matplotlib.pyplot as plt

# scipy imports
import numpy as np
from scipy.stats import poisson

In [62]:
contigs_path = "./contigs/"
reads_path = "../Assembly/Sampled Reads/sd_0004.fastq"


paths = Path(contigs_path).glob('**/*')
contigs_coverages = []

In [63]:
def nomalize(data):
    data = np.array(data).astype(np.float)
    total = np.sum(data)
    
    
    
    def map_f(item):
        return item/total
    
    return np.array(list(map(map_f,data)))

def plot_hist(data):
    # An "interface" to matplotlib.axes.Axes.hist() method
    n, bins, patches = plt.hist(x=data, bins='auto', color='#0504aa',
                                alpha=0.7, rwidth=0.85)
    plt.grid(axis='y', alpha=0.75)
    plt.xlabel('Coverage')
    plt.ylabel('Frequency')
    plt.title('Dsitribution of Coverage')
    plt.show()
    plt.figure()

def detect_links_to_investigate(graph, contig_coverages, link_coverages):
    # if the contig_coverage/read_coverage is 1 and there are multiple outgoing/incoming edges
    # if the contig_coverage/read_coverage is less than the sum of that of out going edges => bogus edge exist
    
    return None

In [64]:
# for contigs
for p in [str(p) for p in paths]:
    print("INFO:: Checking contig", p)
    !minimap2 -a "$p" "$reads_path" > out.txt
    !samtools sort out.txt > out.sorted.txt
    !samtools depth out.sorted.txt > out.txt 
    
    covereage_vals = !cut -f3 out.txt
    
    tot = sum(list(map(int, covereage_vals)))
    
    !rm out.txt out.sorted.txt
    
    contigs_coverages.append({'name': p, 'coverages': covereage_vals})

INFO:: Checking contig contigs/contig_3
[M::mm_idx_gen::0.059*0.99] collected minimizers
[M::mm_idx_gen::0.072*1.33] sorted minimizers
[M::main::0.072*1.33] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.076*1.31] mid_occ = 3
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.079*1.29] distinct minimizers: 372580 (99.87% are singletons); average occurrences: 1.001; average spacing: 5.361
[M::worker_pipeline::10.131*2.80] mapped 19193 sequences
[M::main] Version: 2.14-r886-dirty
[M::main] CMD: minimap2 -a contigs/contig_3 ../Assembly/Sampled Reads/sd_0004.fastq
[M::main] Real time: 10.136 sec; CPU: 28.336 sec; Peak RSS: 0.495 GB
INFO:: Checking contig contigs/contig_2
[M::mm_idx_gen::0.040*1.02] collected minimizers
[M::mm_idx_gen::0.050*1.41] sorted minimizers
[M::main::0.050*1.41] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.054*1.38] mid_occ = 3
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 

In [66]:
# for links
links_path = "./links/"
paths = Path(links_path).glob('**/*')

for p in [str(p) for p in paths]:
    print("INFO:: Checking link", p)
    !minimap2 -a "$p" "$reads_path" > out.txt
    !samtools sort out.txt > out.sorted.txt
    !samtools depth out.sorted.txt > out.txt 
    
    covereage_vals = !cut -f3 out.txt

    !rm out.txt out.sorted.txt
    
    contigs_coverages.append({'name': p, 'coverages': covereage_vals})

INFO:: Checking link links/contig_1+contig_1+
[M::mm_idx_gen::0.001*1.77] collected minimizers
[M::mm_idx_gen::0.001*2.23] sorted minimizers
[M::main::0.001*2.22] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.002*2.14] mid_occ = 2
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.002*2.08] distinct minimizers: 1876 (100.00% are singletons); average occurrences: 1.000; average spacing: 5.330
[M::worker_pipeline::2.536*2.28] mapped 19193 sequences
[M::main] Version: 2.14-r886-dirty
[M::main] CMD: minimap2 -a links/contig_1+contig_1+ ../Assembly/Sampled Reads/sd_0004.fastq
[M::main] Real time: 2.536 sec; CPU: 5.789 sec; Peak RSS: 0.488 GB
INFO:: Checking link links/contig_2-contig_3+
[M::mm_idx_gen::0.001*1.80] collected minimizers
[M::mm_idx_gen::0.002*2.17] sorted minimizers
[M::main::0.002*2.14] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.002*2.08] mid_occ = 2
[M::mm_idx_stat] kmer size: 15; skip: 10; i

In [75]:
for cov in contigs_coverages:
    np_cov = np.array(cov['coverages'])
    np_cov = cov['coverages'] = np_cov.astype(np.float)
    print ("name", cov['name'], "mean", np_cov.mean(), "std", np_cov.std())
    

name contigs/contig_3 mean 48.824345946075674 std 7.357649532867539
name contigs/contig_2 mean 51.54790862465534 std 10.38956648320026
name contigs/contig_1 mean 102.2143936 std 9.196484729736632
name links/contig_1+contig_1+ mean 103.7281 std 8.163379838645266
name links/contig_2-contig_3+ mean 77.2774 std 27.446949725607034
name links/contig_1+contig_3+ mean 73.4377 std 23.53335757409044
name links/contig_2-contig_1+ mean 107.567 std 5.825316386257488


In [146]:
print(contigs_coverages[0]['name'])


from scipy.stats import norm

name, data = contigs_coverages[0]["name"], contigs_coverages[0]["coverages"]

xx = norm.fit(data)

print("Considering 2- -> 1+")
'''2- 1+'''
mc_2 = 51.54790862465534
std_c2 = 10.38956648320026

mc_1 = 102.2143936
std_c1 = 9.196484729736632

mc_3 = 48.824345946075674
std_c3 = 7.357649532867539

pl21_c2c1 = norm.pdf(107.567, loc=mc_2,scale=std_c2) * norm.pdf(107.567, loc=mc_1,scale=std_c1)

pl23_c2c3 = norm.pdf(77.2774, loc=mc_2,scale=std_c2) * norm.pdf(107.567, loc=mc_3,scale=std_c3)

pl13_c1c3 = norm.pdf(73.4377, loc=mc_1,scale=std_c2) * norm.pdf(107.567, loc=mc_3,scale=std_c3)


print("Chance of 2-1+ over 2-3+", pl21_c2c1 / (pl21_c2c1 + pl23_c2c3))

print("Chance of 2-3+ over 2-1+", pl23_c2c3 / (pl21_c2c1 + pl23_c2c3))

# print(pl21_c2c1 / (pl21_c2c1 + pl13_c1c3))

# print(pl21_c2c1, pl23_c2c3)
# p1 = poisson.pmf(51.54790862465534, xx[0])


# p1 = norm.pdf(49.5, loc=xx[0],scale=[1])
# print(norm.pdf(50, loc=xx[0],scale=[1]), xx)
# print(poisson.pmf(77, xx[0]))


contigs/contig_3
Considering 2- -> 1+
Chance of 2-1+ over 2-3+ 0.9999999979576455
Chance of 2-3+ over 2-1+ 2.042354573456805e-09
